In [1]:
from base.connect_mysql import Database
from datetime     import datetime
from time import time
import pandas as pd


In [2]:
class InitDatabase:
    def __init__(self):
        self.database_cms = Database(mysql_user="ai_luongtuananh", mysql_password="ns6A4qenwQBh3KHW", mysql_host="10.140.0.190", mysql_database="gapo_cms")
        self.database_main = Database(mysql_user="ai_levantuananh", mysql_password="6B4Zfr6ETv334YGn", mysql_host="10.19.96.49", mysql_database="gapo_api")
    
    def select_data_from_database_cms(self, time_start, time_end):
        results = self.database_cms.select(database="gapo_cms",
                                  table_name="ai_post_label",
                                  columns=['post_id', 'new_label','labeler'],
                                  between_name ="updated_at",
                                  between_values=[time_start,time_end],
                                  key_names=['new_label'],
                                  key_ops = ['is not'],
                                  key_values=[None])
        
        return results
    
    def select_data_from_data_main(self, post_id):
        results = self.database_main.select(database="gapo_api",
                              table_name="tbl_post",
                              columns=['id', 'content','media','label','update_time'],
                              key_names=['id'],
                              key_values=[post_id])
        
        return results

In [3]:
class DataProcess:
    def __init__(self):
        self.database = InitDatabase()
        self.log_time_path = 'log_data_1.txt'
    
    def get_data_from_database_cms(self):
        data = []
        try:
            with open(self.log_time_path) as f:
                time_start = int(f.read().split("|")[0])
        except:
            time_start = 1618246800
        
        time_end = time_start + 3600
#         time_break = int(time())
        time_break = 1618333200
        while True:
            if time_start > time_break:
                print('out of data')
                break
            
            info_post_in_database_cms = self.database.select_data_from_database_cms(time_start, time_end)
            data += info_post_in_database_cms
            
            time_start = time_end
            time_end   = time_start+3600
            date       = str(time_start)+"|"+str(datetime.fromtimestamp(int(time_start)))
            print('Data', str(datetime.fromtimestamp(int(time_start))))
            
            with open(self.log_time_path, 'w') as f:
                f.write(str(date))
        
        return data
    
    def get_data_from_database_main(self, post_ids):
        data = []
        for post_id in post_ids:
            info_post_in_database_main =  self.database.select_data_from_data_main(post_id)
            data += info_post_in_database_main
        
        return data
    
    def process(self):
        data_in_database_cms = self.get_data_from_database_cms()
        
        post_ids = []
        post_ids_dict = {}
        for i in data_in_database_cms:
            post_ids.append(i[0])
            post_ids_dict[i[0]] = i
        
        data_in_database_main = self.get_data_from_database_main(post_ids)
        
        infos_post_raw = []
        for data in data_in_database_main:
            post_id = data[0]
            content = data[1]
            media = data[2]
            label = data[3]
            
            update_time = data[4]
            cms_info_of_post_id = post_ids_dict[post_id]
            labeler = cms_info_of_post_id[2]
            value = (post_id, content, media, label, labeler, update_time)
            
            infos_post_raw.append(value)
        

        infos_post_raw_df = pd.DataFrame(infos_post_raw, 
                                         columns=['id', 'content', 'media', 'label', 'labeler', 'update_time'])
        return infos_post_raw_df
        
            

In [4]:
post_raw_df = DataProcess().process()

Connect to MYSQL Server version  8.0.20
Connect to MYSQL Server version  5.7.33-google-log
Data 2021-04-13 01:00:00
Data 2021-04-13 02:00:00
Data 2021-04-13 03:00:00
Data 2021-04-13 04:00:00
Data 2021-04-13 05:00:00
Data 2021-04-13 06:00:00
Data 2021-04-13 07:00:00
Data 2021-04-13 08:00:00
Data 2021-04-13 09:00:00
Data 2021-04-13 10:00:00
Data 2021-04-13 11:00:00
Data 2021-04-13 12:00:00
Data 2021-04-13 13:00:00
Data 2021-04-13 14:00:00
Data 2021-04-13 15:00:00
Data 2021-04-13 16:00:00
Data 2021-04-13 17:00:00
Data 2021-04-13 18:00:00
Data 2021-04-13 19:00:00
Data 2021-04-13 20:00:00
Data 2021-04-13 21:00:00
Data 2021-04-13 22:00:00
Data 2021-04-13 23:00:00
Data 2021-04-14 00:00:00
Data 2021-04-14 01:00:00
out of data


DatabaseError: 2003 (HY000): Can't connect to MySQL server on '10.19.96.49:3306' (110)

In [ ]:
post_raw_df.to_csv('/home/phuongdx/python/GAPO/data/tbl_post_19_02_2021_to_28_02_raw.csv')

In [ ]:
post_raw_df.shape